In [ ]:
import pandas as pd
import json
from pandas import Series,DataFrame
import re
import datetime
from bs4 import BeautifulSoup
import requests
#寻找小区对应的区域和建造年份
def get_data(x):
    try:
        html=requests.get('https://dl.lianjia.com/xiaoqu/rs'+x)
        soup=BeautifulSoup(html.text,'lxml')
        s=soup.select('.info .title a')
        for i in range(0,len(s)):
            if s[i].string==x:
                district=soup.select('.info .positionInfo .district')[i].string
                bizcircle=soup.select('.info .positionInfo .bizcircle')[i].string
                temp=soup.select('.info .positionInfo')[i].get_text()
                nianfen=re.search('\d+',temp).group()
                return x+' '+district+' '+bizcircle+' '+ str(nianfen)
        else:
            return None
    except IndexError as e:
        print(x)

pd.set_option('display.max_rows',None)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',None)

item=[]
for line in open('data/lianjia.json','r',encoding='utf-8'):
    item.append(json.loads(line))
    
df=DataFrame(item)
df.drop_duplicates(inplace=True)
#把车位这一行删掉
df.drop(index=df[df['area']=='车位'].index,inplace=True) 

In [ ]:
getarea=lambda x: float(re.search(r"\d+(\.\d+)?",x).group())
df['area']=df['area'].apply(getarea)
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))
df['guaPai']=df['guaPai'].apply(getarea)

In [ ]:
#找出降价幅度大的房子
f=df[((df['guaPai']-df['totalPrice'])/df['guaPai']>0.05) & (df['dealDate'].map(lambda x:x.year)==2021)].sort_values(by=['title','dealDate'])
f['chazhi']=round((f['guaPai']-f['totalPrice'])/f['guaPai'],2)

In [ ]:
#找出各个销售的销售量，目前看工人村单个销售卖的还真多
s=df.groupby(['seller'])[['seller']].count()
s.index.name='num'
s.sort_values(by=['seller'],ascending=False)
s.head()

In [ ]:
#找出各小区的销售数量，先忽略时间
# xiaoqu=df.groupby(['title'])[['title']].count()
xiaoqu=df['title'].groupby([df['title'],df['dealDate'].apply(lambda x:x.year)]).count()
xiaoqu.index.names=['title','year']
result=list(set(list(map(lambda x:re.search('悦泰(\w+)?',x),xiaoqu.index.levels[0]))))
result.remove(None)
r=[i.group() for i in result]


In [ ]:
s1=xiaoqu.loc[r[1]]
s2=xiaoqu.loc[r[0]]
s1.name=r[1]
s2.name=r[0]
print(s1,s2)


In [ ]:
#找出某个小区的销售情况
print(get_data('第五郡五号地'))
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)
func=lambda x:str((x.year)*100+(x.month))[0:4]+'-'+str((x.year)*100+(x.month))[4:]
df_wu['month']=df_wu['dealDate'].map(func)
df_wu['title'].groupby(df_wu['month']).count()
# df[df['title']=='金海花园西园'][['title']].count()

In [ ]:
#找出小区每年均价，按月标本太少，不太准确
df_wu['unitPrice'].groupby(df_wu['dealDate'].map(lambda x:x.year)).describe()

In [ ]:
#找出小区的成交周期
df_wu['dealCycle'].groupby(df_wu['dealDate'].apply(lambda x:x.year)).describe()

In [ ]:
#找出房子里只有一天成交周期的房子
df[(df['dealCycle'].map(lambda x:x==1)) & (df['dealDate'].apply(lambda x:x.year)==2021)].head()

In [ ]:
#备用功能        
# list=[]
# for i in df['title'].drop_duplicates():
#     g=get_data(i)
#     if g is not None:
#         list.append(g)

# %run main.py
#df.to_json(path_or_buf='data/test.json',orient='index',force_ascii=False)
#检查area数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
#         print(df.iloc[i])
#         df.drop([i])
# #检查挂牌数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search('挂牌*',result[7]) is None:
#         print(str(i))
        # df.drop([i])

In [ ]:
#根据已有数据算全部小区
dd=df['unitPrice'].groupby(df['dealDate'].apply(lambda x:x.year)).describe()
dd[dd.index>2015]

In [ ]:
import json
from pandas import DataFrame
# def get_xiaoqu(n1, n2):
list = []
for line in open('data/lianjia.json', 'r', encoding='utf-8'):
    list.append(json.loads(line))


df = DataFrame(list)  
l=[x for x in df['title']]
s=[x for x in set(l) if x is not None]
s.sort()
item = s[0:1]
# return item

# get_xiaoqu(0,15)

In [3]:
from spider_lianjia.spiders.chengjiao import get_chengjiao_one
g=get_chengjiao_one()
g.get_chengjiao('悦泰福里')

AttributeError: 'list' object has no attribute 'get'